In [15]:
import numpy as np #For matrix operations
import matplotlib.pyplot as plt #For plotting
import mpmath as mp #For hypergeometric function
import scipy.special as ssp #For special functions like gamma, etc.
import qutip as qt #For Quantum Mechanical Calculations
from tabulate import tabulate #For making table

In [23]:
l = 0  # Azimuthal Quantum Number

bet = 0.72291  # In GeV, Value of Beta
#C = 0.2855833  # In GeV, C = V_0 - V_C
C = 0.2858833
w = 0.2021667  # In GeV, frequency in natural units.

K = 0.52  # In GeV
a = 2.34  # In GeV^{-1}

mb = 5.17  # GeV
mu = mb / 2  # Reduced mass

In [24]:
#Conbination Function
def nCk(n1, k1):
    return ssp.gamma(n1 + 1) / (ssp.gamma(k1 + 1) * ssp.gamma(n1 - k1 + 1))

In [25]:
#Kronecker Delta Function
def kron_del(x, y):
    if x == y:
        return 1
    else:
        return 0

In [26]:
#Setting Value of Number of states to sum over
i0 = 40

#Creating some empty matrices
H_mn0 = np.zeros([i0, i0])
H_mn1 = np.zeros([i0, i0])
H_mn2 = np.zeros([i0, i0])
H_mn3 = np.zeros([i0, i0])
H_mn = np.zeros([i0, i0])
nv2 = np.zeros([i0, i0])

In [27]:
for i in range(i0):
    n = i
    N_nl = np.sqrt(2 * (bet ** 3) * ssp.factorial(n) / ssp.gamma(n + l + 1.5)) #Normalization Constant

    for j in range(i0):
        m = j
        N_ml = np.sqrt(2 * (bet ** 3) * ssp.factorial(m) / ssp.gamma(m + l + 1.5))  #Normalization Constant

        #x = 1
        #y = 1
        
        #HyperGeometric Part of Integral
        def g1(x):
            return mp.hyp3f2((l+1),-m,-n, (0.5-m),(0.5-n), x)

        def g2(x):
            return mp.hyp3f2((l+2),-m,-n,(1.5-m),(1.5-n),x)
        
        #Calculating H_mn_i and storing them in empty matrices
        H_mn0[i,j] = C * kron_del(m, n)
        
        H_mn1[i,j] = -((K / (2 * (bet ** 2)))*ssp.gamma(l + 1)*(N_nl * N_ml)*( nCk((n - 0.5), n) * nCk((m - 0.5), m)) * g1(1))
        
        H_mn2[i,j] = ((1 / (2 * (a**2)*(bet ** 4)))*ssp.gamma(l + 2)*(N_nl * N_ml)*(nCk((n - 1.5), n)*nCk((m - 1.5), m))*g2(1))
        
        H_mn3[i,j] = (-mu * (w ** 2) / (4 * (bet ** 5)))*(N_nl * N_ml)*(((-ssp.gamma(n + l + 2.5) / ssp.gamma(n + 1)) * (kron_del(m, n+1))) 
                    + ((((2*n) + l + 1.5)*ssp.gamma(n + l + 1.5) / ssp.gamma(n + 1))*(kron_del(m, n))) 
                    + (((-ssp.gamma(n + l + 1.5) / ssp.gamma(n)) * (kron_del(m, n-1)))))
        
        #Calculating H_mn
        H_mn[i, j] = H_mn0[i,j] + H_mn1[i,j] + H_mn2[i,j] + H_mn3[i,j]

        #Calculating and storing the terms G_mn (defined in my code) in nv2 matrix
        if i == j: 
            nv2[i, j] = 0
        else:
            nv2[i, j] = (((H_mn[i, j]) ** 2) / (2 * (n - m) * w))


In [28]:
#Calculating the Energy corrections, and comparing the net result from actual energy eigenvalues
header = ["E_n^{(0)}", "E_n^{(1)}", "E_n^{(2)}", "E_n^{(0)} + E_n^{(1)} + E_n^{(2)}", "Actual Energies (E_n)"]

Col1 = np.array([9.64667341, 10.0510042,  10.45532851, 10.85964687])
Col2 = np.array([H_mn[i, i] for i in range(4)])
Col3 = np.array([np.sum(nv2[i,:]) for i in range(4)])

Col4 = Col1 + Col2 + Col3
Col5 = np.array([9.45999529, 10.05155864, 10.39672205, 10.67363567])

zipped_lists = zip(Col1, Col2, Col3, Col4, Col5)

#Printing List
print(tabulate(zipped_lists, header, tablefmt="fancy_grid"))

╒═════════════╤═════════════╤═════════════╤═════════════════════════════════════╤═════════════════════════╕
│   E_n^{(0)} │   E_n^{(1)} │   E_n^{(2)} │   E_n^{(0)} + E_n^{(1)} + E_n^{(2)} │   Actual Energies (E_n) │
╞═════════════╪═════════════╪═════════════╪═════════════════════════════════════╪═════════════════════════╡
│     9.64667 │ -0.00485309 │  -0.114004  │                             9.52782 │                  9.46   │
├─────────────┼─────────────┼─────────────┼─────────────────────────────────────┼─────────────────────────┤
│    10.051   │  0.00620579 │  -0.0194791 │                            10.0377  │                 10.0516 │
├─────────────┼─────────────┼─────────────┼─────────────────────────────────────┼─────────────────────────┤
│    10.4553  │ -0.050181   │  -0.0146411 │                            10.3905  │                 10.3967 │
├─────────────┼─────────────┼─────────────┼─────────────────────────────────────┼─────────────────────────┤
│    10.8596  │ -0.137261   